In [1]:
import numpy as np
import awkward as ak
import uproot_methods

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [3]:
def stack_arrays(a, keys, axis=-1):
    flat_arr = np.stack([a[k].flatten() for k in keys], axis=axis)
    return awkward.JaggedArray.fromcounts(a[keys[0]].counts, flat_arr)

In [4]:
def pad_array(a, maxlen, value=0., dtype='float32'):
    x = (np.ones((len(a), maxlen)) * value).astype(dtype)
    for idx, s in enumerate(a):
        if not len(s):
            continue
        trunc = s[:maxlen].astype(dtype)
        x[idx, :len(trunc)] = trunc
    return x

In [5]:
##and Professor suggests that we could use mass, classifacation for later application
def SetAKArr(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    n_particles_ls = []
    px_ls = []
    py_ls = []
    pz_ls = []
    energy_ls = []
    mass_ls = []
    charge_ls = []
    _label1 = []
    _label2 = []
    _label3 = []
    _label4 = []
    _label5 = []
    
    n = 0
    for line in lines:
        if line.startswith('E'):
            if not n == 0:
                n_particles_ls.append(n)
                n = 0
            exp_inf = line.split()
#             _label1.append(int(exp_inf[1]))
#             _label2.append(1-int(exp_inf[1]))
#             _label1.append(1)
#             _label2.append(0)
            _label1.append(float(exp_inf[1]))
            _label2.append(float(exp_inf[2]))
            _label3.append(float(exp_inf[3]))
            _label4.append(float(exp_inf[4]))
            _label5.append(float(exp_inf[5]))
        else:
            par = line.split()
            ##particle +1
            n = n + 1
            px_ls.append(abs(float(par[2])))
            py_ls.append(abs(float(par[3])))
            pz_ls.append(abs(float(par[4])))
            energy_ls.append(abs(float(par[5])))
            mass_ls.append(abs(float(par[6])))  
            charge_ls.append(int(par[0]))
#             px_ls.append(6)
#             py_ls.append(2)
#             pz_ls.append(3)
#             energy_ls.append(4)
#             mass_ls.append(5)
    if not n == 0:
        n_particles_ls.append(n)
    px_arr = np.array(px_ls)
    py_arr = np.array(py_ls)
    pz_arr = np.array(pz_ls)
    energy_arr = np.array(energy_ls)
    mass_arr = np.array(mass_ls)
    charge_arr = np.array(charge_ls)
    n_particles = np.array(n_particles_ls)

#     print(n_particles)
    px = ak.JaggedArray.fromcounts(n_particles, px_arr)
    py = ak.JaggedArray.fromcounts(n_particles, py_arr)
    pz = ak.JaggedArray.fromcounts(n_particles, pz_arr)
    energy = ak.JaggedArray.fromcounts(n_particles, energy_arr)
    mass = ak.JaggedArray.fromcounts(n_particles, mass_arr)
    charge = ak.JaggedArray.fromcounts(n_particles, charge_arr)
    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    ##Create an Order Dic
    from collections import OrderedDict
    v = OrderedDict()
    v['part_px'] = px
#     print(px)
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy
    v['part_mass'] = mass
    v['charge'] = charge
    v['part_e_log'] = np.log(energy)
    v['part_px_log'] = np.log(px)
    v['part_py_log'] = np.log(py)
    v['part_pz_log'] = np.log(pz)
    v['part_m_log'] = np.log(mass)
#     ls1 = [1,2,3,4]
#     ls2 = [5,6,7,8]
#     v['label'] = np.stack((_label1, _label2, _label3, _label4, _label5), axis=-1)
#     print(v['label'])
    v['label'] = np.stack((_label4, _label5), axis = -1)
#     v['label'] = np.stack(_label1, axis = -1)
#     print(v['label'])
    return v

In [6]:
class Dataset(object):
    def __init__(self, filepath, feature_dict = {}, label = 'label', pad_len=100, data_format='channel_first'):
        self.filepath = filepath
        self.feature_dict = feature_dict
        if len(feature_dict) == 0:
            feature_dict['points'] = ['part_px','part_py','part_pz']
            feature_dict['features'] = ['part_energy', 'part_mass', 'charge', 'part_px', 'part_py', 'part_pz']
            feature_dict['mask'] = ['part_energy']
        ##currently we use 'E' for experiments
        self.label = label
        self.pad_len = pad_len
        assert data_format in ('channel_first', 'channel_last')
        self.stack_axis = 1 if data_format=='channel_first' else -1
        self._values = {}
        self._label = None
        self._load()
        
    def _load(self):
        logging.info('Start loading file %s' % self.filepath)
#         counts = None
        a = SetAKArr(self.filepath)
        self._label = a[self.label]
        for k in self.feature_dict:
                cols = self.feature_dict[k]
                if not isinstance(cols, (list, tuple)):
                    cols = [cols]
                arrs = []
                for col in cols:
                    arrs.append(pad_array(a[col], self.pad_len))
                    ##check the dimesion of a[col], and it should be array.
                self._values[k] = np.stack(arrs, axis=self.stack_axis)
        logging.info('Finished loading file %s' % self.filepath)
        
        
        
    def __len__(self):
        return len(self._label)

    def __getitem__(self, key):
        if key==self.label:
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]

In [7]:
train_dataset = Dataset('train.txt', data_format='channel_last')
val_dataset = Dataset('val.txt', data_format='channel_last')

[2024-03-02 13:56:02,927] INFO: Start loading file train.txt
/home/htk/miniforge3/envs/tensorflow/lib/python3.5/site-packages/awkward/array/jagged.py:976: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
[2024-03-02 13:56:03,148] INFO: Finished loading file train.txt
[2024-03-02 13:56:03,149] INFO: Start loading file val.txt
[2024-03-02 13:56:03,181] INFO: Finished loading file val.txt


[[5.28917911 5.27933   ]
 [5.28466195 5.27933   ]
 [5.2930648  5.27933   ]
 ...
 [5.29284146 5.27933   ]
 [5.30349918 5.27933   ]
 [5.28053101 5.27933   ]]
[[5.29431961 5.27933   ]
 [5.28574823 5.27933   ]
 [5.29154575 5.27933   ]
 [5.30078546 5.27933   ]
 [5.29259698 5.27933   ]
 [5.28524332 5.27933   ]
 [5.29155483 5.27933   ]
 [5.27941453 5.27933   ]
 [5.29880525 5.27933   ]
 [5.29218013 5.27933   ]
 [5.29228723 5.27933   ]
 [5.28728002 5.27933   ]
 [5.28741552 5.27933   ]
 [5.2891755  5.27933   ]
 [5.28565425 5.27933   ]
 [5.29274715 5.27933   ]
 [5.2994644  5.27933   ]
 [5.30219958 5.27933   ]
 [5.3852338  5.27933   ]
 [5.28510522 5.27933   ]
 [5.29839353 5.27933   ]
 [5.28828447 5.27933   ]
 [5.29024063 5.27933   ]
 [5.28869517 5.27933   ]
 [5.2903763  5.27933   ]
 [5.31020409 5.27933   ]
 [5.29404419 5.27933   ]
 [5.28998162 5.27933   ]
 [5.29208506 5.27933   ]
 [5.28857468 5.27933   ]
 [5.28599418 5.27933   ]
 [5.28998564 5.27933   ]
 [5.30079814 5.27933   ]
 [5.29873706 5.2793

In [8]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [9]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
##this shows the number of classes for classification
num_classes = train_dataset.y.shape[1]
# num_classes = 1
# print(num_classes)
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

In [10]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 30

In [11]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [12]:
# model.compile(loss='categorical_crossentropy',
#               optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])
model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

[2024-03-02 13:56:16,808] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_NotEqual (TensorFlo [(None, 100, 1)]     0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_Cast (TensorFlowOpL [(None, 100, 1)]     0           tf_op_layer_NotEqual[0][0]       
__________________________________________________________________________________________________
tf_op_layer_Equal (TensorFlowOp [(None, 100, 1)]     0           tf_op_layer_Cast[0][0]           
________________________________________________________________________________________

In [13]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler]

In [14]:
train_dataset.shuffle()
model.fit(train_dataset.X, train_dataset.y,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=15, # --- train only for 1 epoch here for demonstration ---
          validation_data=(val_dataset.X, val_dataset.y),
          shuffle=True,
          callbacks=callbacks)

[2024-03-02 13:56:19,094] INFO: Learning rate: 0.001000


Epoch 1/15
5/5 [==============================] - ETA: 0s - accuracy: 0.7631 - loss: 28.3624

[2024-03-02 13:56:25,574] WARNING: Can save best model only with val_acc available, skipping.


5/5 [==============================] - 4s 837ms/step - accuracy: 0.7631 - loss: 28.3624 - val_accuracy: 0.5190 - val_loss: 27.4525


[2024-03-02 13:56:25,578] INFO: Learning rate: 0.001000


...

5/5 [==============================] - 4s 814ms/step - accuracy: 0.5043 - loss: 0.2606 - val_accuracy: 0.0000e+00 - val_loss: 19.9696


[2024-03-02 13:57:33,618] INFO: Learning rate: 0.000100


Epoch 15/15
5/5 [==============================] - ETA: 0s - accuracy: 0.5061 - loss: 0.2457

[2024-03-02 13:57:38,909] WARNING: Can save best model only with val_acc available, skipping.


5/5 [==============================] - 4s 813ms/step - accuracy: 0.5061 - loss: 0.2457 - val_accuracy: 0.0000e+00 - val_loss: 19.8490


In [16]:
predictions = model.predict(val_dataset.X)
for prediction in predictions:
    print(prediction)

[0.81566733 0.8419997 ]
[0.7763244 0.8333638]
[0.8814611  0.94231457]
[0.8059118  0.84061307]
[0.800409  0.8586278]
[0.794723  0.8513902]
[0.8450996 0.8829296]
[0.7606166  0.78812855]
[0.83673567 0.8708411 ]
[0.797791   0.83190423]
[0.8492246  0.87396985]
[0.76914084 0.81507653]
[0.7250046 0.7833844]
[0.8454234  0.89155895]
[0.8084389  0.84663886]
[0.76829237 0.79752666]
[0.8014586 0.8500824]
[0.82727224 0.85501397]
[0.84449154 0.893961  ]
[0.87033206 0.91251904]
[0.78092766 0.83082753]
[0.8494279  0.88037974]
[0.7671171  0.83008164]
[0.83558804 0.8672809 ]
[0.7874028  0.82682735]
[0.8468027 0.8712953]
[0.82483774 0.8575669 ]
[0.8609012  0.88719565]
[0.8809435 0.9194246]
[0.81847644 0.8526326 ]
[0.84798557 0.88274235]
[0.73060113 0.7845528 ]
[0.8040942 0.8346309]
[0.78252846 0.8210624 ]
[0.81995493 0.8528581 ]
[0.80983704 0.8465799 ]
[0.87252754 0.9165501 ]
[0.85170454 0.89759773]
[0.72482663 0.76189774]
[0.7590944 0.8161345]
[0.896444   0.94038475]
[0.7336928 0.7760537]
[0.7311706 0.7